In [1]:
import pandas as pd
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
import collections
import math

warnings.filterwarnings('ignore')

In [2]:
path = 'E:/Dataset/新闻推荐比赛数据/'
train = path + 'articles.csv'
train_click = path + 'train_click_log.csv'

In [3]:
# 使用历史浏览 点击文章的数据信息预测用户未来的点击行为，即用户最后一次点击新闻的文章
# 问题转换 30万用户 200w点击 36w篇文章
train_df = pd.read_csv(train)
train_click_df = pd.read_csv(train_click)

In [4]:
train_df

,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162
...,...,...,...,...
364042,364042,460,1434034118000,144
364043,364043,460,1434148472000,463
364044,364044,460,1457974279000,177
364045,364045,460,1515964737000,126


In [4]:
train_click_df.memory_usage().sum()/1024**2

76.39788055419922

In [5]:
def reduce_memory(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_types = df[col].dtypes
        if col_types in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_types)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem, 
                                                                                                    100*(start_mem-end_mem)/start_mem, 
                                                                                                    (time.time()-starttime)/60))
    return df

In [6]:
# 定义
user_recall_items_dict = collections.defaultdict(dict)
user_recall_items_dict

defaultdict(dict, {})

In [7]:
import copy
all_click_df = copy.deepcopy(train_click_df)
def get_user_item_time(all_click_df):
    c = all_click_df.sort_values('click_timestamp')
    # 这应该是个键值对list(zip(c['click_article_id'], c['click_timestamp']))
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    user_item_time_df = c.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x)).reset_index().rename(columns={0:'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    return user_item_time_dict
# get_user_item_time(all_click_df)

In [20]:
def get_item_topk_click(click_df, k):
    topk_list = click_df['click_article_id'].value_counts().index[:k]
    return topk_list
item_topk_click = get_item_topk_click(all_click_df, 10)

In [9]:
user_item_time_dict = get_user_item_time(all_click_df)
# 计算物品相似度
i2i_sim = {}
item_cnt = defaultdict(int)
ii = 0
for user, item_time_list in tqdm(user_item_time_dict.items()):
#     print(user, item_time_list)
#     print('-'*10)
    # 基于商品的协同过滤
    for i, i_click_time in item_time_list:
#         print(i, i_click_time)
        item_cnt[i] += 1
        i2i_sim.setdefault(i, {})  # 当key不存在时，设置为默认值，key存在时，返回值
        for j, j_click_time in item_time_list:
            if(i == j):
                continue
            i2i_sim[i].setdefault(j,0)
            i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)  # 1/log(n+1)
            ii += 1
            
i2i_sim_ = i2i_sim.copy()
for i, related_items in i2i_sim.items():
    print(i, related_items)
    for j, wij in related_items.items():
        i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])  # 
#     i += 1
#     if i == 10:
#         break

100%|███████████████████████████████████████████████████████████████████████| 200000/200000 [00:11<00:00, 17676.17it/s]
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




10500 {161506: 0.48089834696298783, 62567: 0.833854470827749, 8224: 2.0970299708553384, 39894: 0.48089834696298783, 181668: 0.48089834696298783, 9960: 0.48089834696298783, 8222: 2.9285897662948424, 8223: 2.3374791443368323, 203890: 0.45511961331341866, 124749: 0.45511961331341866, 75345: 0.45511961331341866, 214753: 0.45511961331341866, 199372: 0.45511961331341866, 61927: 0.45511961331341866, 36172: 0.45511961331341866, 293062: 1.072008968920998, 124194: 0.5581106265512472, 118391: 0.5581106265512472, 284026: 0.6213349345596119, 123289: 0.6213349345596119, 198144: 0.6213349345596119, 235028: 0.43429448190325176, 168612: 0.43429448190325176, 114664: 0.43429448190325176, 9188: 0.43429448190325176, 9818: 0.43429448190325176, 8296: 0.43429448190325176, 9120: 0.43429448190325176, 8831: 0.43429448190325176, 66452: 0.35295612386476116, 129029: 0.35295612386476116, 36709: 0.35295612386476116, 36622: 0.35295612386476116, 124228: 0.35295612386476116, 276970: 0.35295612386476116, 70457: 0.352956

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



288243 {156624: 0.318928988903801, 160417: 0.318928988903801, 341339: 0.318928988903801, 342380: 0.318928988903801, 141004: 0.318928988903801, 87224: 0.318928988903801, 161506: 0.318928988903801, 70830: 0.318928988903801, 236541: 0.318928988903801, 158040: 0.318928988903801, 235525: 0.318928988903801, 233658: 0.318928988903801, 288320: 0.318928988903801, 337442: 0.318928988903801, 351517: 0.318928988903801, 332522: 0.318928988903801, 326592: 0.637857977807602, 327959: 0.318928988903801, 286202: 0.318928988903801, 353285: 0.318928988903801}
166362 {165352: 0.9102392266268373}
165352 {166362: 0.9102392266268373}
348609 {288410: 0.43429448190325176, 285343: 0.43429448190325176, 124734: 0.43429448190325176, 140985: 0.43429448190325176, 124194: 0.43429448190325176, 123290: 0.43429448190325176, 348601: 0.43429448190325176, 288435: 0.43429448190325176}
111088 {272143: 0.5138983423697507, 156447: 0.5138983423697507, 111031: 0.5138983423697507, 16129: 0.5138983423697507, 118357: 0.5138983423697

In [ ]:
user_item_time_dict

In [ ]:
# itemcf 的物品相似度计算
def itemcf_sim(df):
    """
    文章与文章的相似性计算
    return: 文章与文章的相似性矩阵
    思路：基于物品的协同过滤
    """
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.item())

In [ ]:
# 获取点击最多的topk个文章
topk_click = click_df['click_article_id'].value_counts()

In [ ]:
{30760: {157507: 0.01076245713904683, 211442: 0.05262861609293018}}

In [ ]:
# 根据点击时间获取用户的点击文章序列 {user1: [(item1, time1), (item2, item2)..]..}
def get_user_item_time(click_df):
    

In [43]:
user_item_time_dict

{0: [(30760, 1508211672520), (157507, 1508211702520)],
 1: [(289197, 1508211316889), (63746, 1508211346889)],
 2: [(36162, 1508211438695), (168401, 1508211468695)],
 3: [(50644, 1508211359672), (36162, 1508211389672)],
 4: [(42567, 1508211625466), (39894, 1508211655466)],
 5: [(211442, 1508211243884), (234481, 1508211273884)],
 6: [(62464, 1508212140776), (10023, 1508212170776)],
 7: [(50644, 1508211313878), (211442, 1508211343878)],
 8: [(70986, 1508211172391), (50644, 1508211202391)],
 9: [(70986, 1508211213614), (211442, 1508211245547), (211455, 1508211275547)],
 10: [(50644, 1508211631144), (159195, 1508211661144)],
 11: [(50644, 1508211074535), (234481, 1508211104535)],
 12: [(211442, 1508211061254), (211455, 1508211091254)],
 13: [(36162, 1508211428226), (277107, 1508211458226)],
 14: [(70986, 1508210959082), (36162, 1508210989082)],
 15: [(277107, 1508210921971),
  (342473, 1508211137084),
  (206415, 1508211167084)],
 16: [(50644, 1508210982166), (211442, 1508211012166)],
 17: [

In [ ]:
# itemcf 的物品相似度计算
def itemcf_sim(df):
    """
    文章与文章的相似性计算
    return: 文章与文章的相似性矩阵
    思路：基于物品的协同过滤
    """
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.item())

In [25]:
# 获取点击最多的topk个文章
topk_click = click_df['click_article_id'].value_counts()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
3,199998,157770,1507029532200,4,1,17,1,25,5
45,199987,272143,1507029551778,4,1,17,1,24,5
52,199984,70594,1507029553199,4,3,2,1,5,7
32,199990,272143,1507029564434,4,1,17,1,7,5
0,199999,160417,1507029570190,4,1,17,1,13,1
...,...,...,...,...,...,...,...,...,...
705656,121158,224148,1509798422502,4,1,17,1,13,2
925954,70254,207672,1510093882860,4,1,17,1,20,2
925955,70254,96333,1510093912860,4,1,17,1,20,2
1091177,2465,203538,1510603424886,4,1,17,1,2,2


In [ ]:
{30760: {157507: 0.01076245713904683, 211442: 0.05262861609293018}}

In [ ]:
# 根据点击时间获取用户的点击文章序列 {user1: [(item1, time1), (item2, item2)..]..}
def get_user_item_time(click_df):
    

In [21]:
# 基于itemcf的文章推荐
user_id = 1
user_hist_item = user_item_time_dict[user_id]  # 读过的文章
user_hist_items_ = {user_id for user_id, _ in user_hist_item}  # 读过文章的 article_id

item_rank = {}
# the same as article_id
for loc, (i, click_time) in enumerate(user_hist_item):
    print(loc, (i, click_time))
    for article_id, same_value in sorted(i2i_sim[289197].items(), key=lambda x: x[1], reverse=True)[:5]:
        if article_id in user_hist_items_:
            continue
        item_rank.setdefault(article_id, 0)
        item_rank[article_id] += same_value

# 不足k个，使用k补齐
if len(item_rank) < 10:
    for k, article_id in enumerate(item_topk_click):
        if article_id in item_rank.keys():
            continue
        item_rank[article_id] = -k-100
        if len(item_rank)  == 10:
            break
item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)
item_rank

0 (289197, 1508211316889)
1 (63746, 1508211346889)


[(49405, 0.19235933878519512),
 (106598, 0.12277107057205394),
 (323677, 0.10843401227272671),
 (30408, 0.1014635699357751),
 (234698, -100),
 (123909, -101),
 (336221, -102),
 (96210, -103),
 (183176, -104),
 (336223, -105)]

In [18]:
item_rank

{49405: 0.19235933878519512,
 106598: 0.12277107057205394,
 323677: 0.10843401227272671,
 30408: 0.1014635699357751}